# Как работает логистическая регрессия в Spark: особенности прогноза
`Логистическая регрессия` – это статистическая модель, которая используется в машинном обучении для прогнозирования вероятности возникновения некоторого события путем построения логистической функции и сравнения этого события с кривой этой функции. В результате формируется ответ в виде вероятности бинарного события: `0` и `1`, где `0` – событие не произошло, `1` – событие произошло.

# Работа с логистической регрессией в Spark

Для того, чтобы начать работу по прогнозу данных, необходимо настроить базовую конфигурацию, импортировав некоторые классы библиотек `Spark MLlib` и `Spark SQL`:

In [1]:
!pip install pyspark

In [2]:
!pip install findspark

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import *

ДАТАСЕТ С ДОМАМИ НА ПРОДАЖУ

В качестве примера мы будем использовать датасет Kaggle, который содержит данные о домах на продажу в Бруклине с 2003 по 2017 года и доступен для скачивания. Он содержит 111 атрибутов (столбцов) и 390883 записей (строк). В атрибуты включены: дата продажи, дата постройки, цена на дом, налоговый класс, соседние регионы, долгота, ширина и др.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os

In [ ]:
os.chdir("/content/drive/My Drive/Colab Notebooks/обработка больших данных/apache_spark/spark_seminar/data")
os.listdir()

Теперь необходимо импортировать входные данные, создав на их основе набор RDD (Resilient Distributed Dataset).

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
data = spark.read.csv(
    '/content/drive/My Drive/Colab Notebooks/обработка больших данных/apache_spark/spark_seminar/data/brooklyn_sales_map.csv',
    inferSchema=True, header=True)

# ГОТОВИМ АТРИБУТ ДЛЯ ПОСЛЕДУЮЩЕЙ БИНАРНОЙ КЛАССИФИКАЦИИ

Допустим, требуется классифицировать налоговый класс на дом (tax_class). Всего имеется 10 таких классов. Поскольку данные распределены неравномерно (например, в классе 1 имеется 198969 записей, а в 3-м — только 18), мы разделим их на 2 категории: те, которые принадлежат классу 1, и остальные. В Python это делается очень просто, нужно просто вызвать метод replace:

In [8]:
by_1 = ['1', '1A', '1B', '1C']
by_others = ['2', '2A', '2B', '2C', '3', '4']
data = data.replace(by_others, '0', ['tax_class'])
data = data.replace(by_1, '1', ['tax_class'])

Кроме того, алгоритмы Machine Learning в PySpark работают с числовым значениями, а не со строками. Поэтому преобразуем значения столбца tax_class в тип int:

In [9]:
data = data.withColumn('tax_class', data.tax_class.cast('int'))

# ПОДБОР ПРИЗНАКОВ И ПРЕОБРАЗОВАНИЕ КАТЕГОРИЙ

Выберем следующие признаки для обучения модели `Machine Learning`: год постройки (**year_of_sale**), цена на дом (**sale_price**) и соседние регионы (**neighborhood**). Последние атрибут является категориальным признаком — в данных имеется `20` соседних регионов. Но опять же все значения этих категорий являются строковыми, поэтому нужно преобразовать их в числовые.

Можно воспользоваться методом replace, как это сделано выше, но придётся сначала извлечь названия всех `20` регионов. А можно использовать специальный класс `StringIndexer`` из PySpark-модуля ML`, который выполнит за нас всю работу. Объект этого класса принимает в качестве аргументов: название атрибута, который нужно преобразовать (inputCol), и название, которое будет иметь преобразованный атрибут (outputCol). Вот так это выглядит в Python:

In [10]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="neighborhood", outputCol="neighborhood_id")
data = indexer.fit(data).transform(data)

Преобразованные категории имеют вид:

In [ ]:
data.groupBy('neighborhood_id').count().show()

Теперь выберем необходимые признаки, а также отбросим строки с пустыми значениями с помощью метода dropna в PySpark:

In [12]:
features = ['year_of_sale', 'sale_price', 'neighborhood_id']
target = 'tax_class'
attributes = features + [target]
sample = data.select(attributes).dropna()

# ВЕКТОРИЗАЦИЯ ПРИЗНАКОВ

Поскольку алгоритмы машинного обучения в PySpark принимают на вход только вектора, то нужно  провести векторизацию. Для преобразования признаков в вектора используется класс VectorAssembler. Объект этого класса принимает в качестве аргументов список с названиями признаков, которые нужно векторизовать (inputCols), и название преобразованного признака (outputCol). После создания объекта VectorAssembler вызывается метод transform.

Для начала выберем в качестве признака для преобразования — цену на дом. Код на Python:

In [13]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['sale_price'],
                            outputCol='features')
output = assembler.transform(sample)

Полученный после векторизации DataFrame выглядит следующим образом:

In [ ]:
output.show(5)

# РАЗДЕЛЕНИЕ ДАТАСЕТА И ОБУЧЕНИЕ МОДЕЛИ

Для решения задач Machine Learning всегда нужно иметь, как минимум, две выборки — обучающую и тестовую. На обучающей мы будем обучать модель, а на тестовой проверять эффективность обученной модели. В PySpark сделать это очень просто, нужно просто вызвать метод randomSplit, который разделит исходный датасет в заданной пропорции. Мы разделим в пропорции 80:20, в Python это выглядит так:

In [15]:
train, test = output.randomSplit([0.8, 0.2])

Теперь воспользуемся логистической регрессией (Logistic Regression) [1], которая есть в PySpark, в качестве алгоритма Machine learning. Для этого нужно указать признаки, на которых модель обучается, и признак, который нужно классифицировать. Мы преобразовали цену на дом (sale price) в вектор под названием features, поэтому именно его и указываем в аргументе:

In [16]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features',
                        labelCol='tax_class')
model = lr.fit(train)

Осталось только получить предсказания. Для этого вызывается метод transform, который принимает тестовую выборку:

In [17]:
predictions = model.transform(test)

Проверим эффективность модели, используя метрику качества. И в этом случае PySpark нас выручает, поскольку у него есть класс BinaryClassificationEvaluator. Нужно лишь указать целевой признак (tax class), а затем вызвать метод evaluate и передать в него наши предсказания. В Python это выглядит так:

In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol='tax_class')
print('Evaluation:', evaluator.evaluate(predictions))

Evaluation: 0.5262600768248258


Как видим, мы получили точность только 52%, что очень мало. Попробуем добавить ещё несколько признаков для обучения.

# ДОБАВЛЕНИЕ ПРИЗНАКОВ

Векторизуем также год постройки (year_of_sale) и соседние регионы (neighborhood_id). Для этого нужно только в VectorAssembler указать выбранные признаки:

In [19]:
features = ['year_of_sale', 'sale_price', 'neighborhood_id']

assembler = VectorAssembler(inputCols=features,
                            outputCol='features')
output = assembler.transform(sample)

In [ ]:
output.show(5)

In [21]:
train, test = output.randomSplit([0.8, 0.2])

In [22]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol='features',
                        labelCol='tax_class')
model = lr.fit(train)

In [23]:
predictions = model.transform(test)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol='tax_class')
print('Evaluation:', evaluator.evaluate(predictions))

Python-код для остальных шагов — разделение на тестовую и обучающую выборки, обучение и оценивание модели — остаётся все тем же. В итоге, мы смогли повысить точность до 60%: